<a href="https://colab.research.google.com/github/lcqsigi/big-data2/blob/main/063_Zillow_For_Sale_Properties_Python3.091724.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
### Get Keys
1) Get Free [Rapid API Key by creating an account](https://rapidapi.com/marketplace)

Store each key in a central and private location.

### Subscribe to APIs

1) Subscribe to [Zillow on Rapid API](https://rapidapi.com/apimaker/api/zillow-com1/)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [1]:
from google.colab import drive, files # specific to Google Colab
import pandas as pd
import requests
import json
import time

# show all columns
pd.set_option('display.max_columns', None)

## <font color="blue">Functions</font>

## <font color="blue">Local & Constants</font>

In [2]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/' # optional

############
# REQURIED #
############
city = 'hayward'
state = 'ca'
search_str = city + ', ' + state
print('Search string:', search_str)

Mounted at /content/drive
Search string: hayward, ca


## <font color="blue">Data</font>

### Get API Keys

In [3]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

In [4]:
rapid_api_key

'8b63d59cecmsha5a796447533ec2p11f828jsnf7fd2c8395a5'

In [5]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

In [6]:
headers = {
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }
import requests
req = requests.request('GET', url, headers=headers)

## <font color="green">Zillow</font>

### <font color="purple">Endpoint: Property</font>

In [7]:
# get data
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring1 = {
               "location": "94611",
               "home_type":"Single_Family",
               "minPrice": "800000",
               "maxPrice": "1150000",
               "sqftMin": "1500",
               "bedsMin": "3"
               }

querystring2 = {
               "location": "94605",
               "home_type":"Single_Family",
               "minPrice": "800000",
               "maxPrice": "1150000",
               "sqftMin": "1500",
               "bedsMin": "3"
               }

querystring3 = {
               "location": "94619",
               "home_type":"Single_Family",
               "minPrice": "800000",
               "maxPrice": "1150000",
               "sqftMin": "1500",
               "bedsMin": "3"
               }

headers = {
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }

z_for_sale_resp1 = requests.request("GET", url, headers=headers, params=querystring1)
z_for_sale_resp2 = requests.request("GET", url, headers=headers, params=querystring2)
z_for_sale_resp3 = requests.request("GET", url, headers=headers, params=querystring3)

# transform to json
z_for_sale_resp_json1 = z_for_sale_resp1.json()
z_for_sale_resp_json2 = z_for_sale_resp2.json()
z_for_sale_resp_json3 = z_for_sale_resp3.json()

In [8]:
len(z_for_sale_resp_json1.get('props', []))

11

In [9]:
# view data
df_z_for_sale1 = pd.json_normalize(data=z_for_sale_resp_json1['props'])
df_z_for_sale2 = pd.json_normalize(data=z_for_sale_resp_json2['props'])
df_z_for_sale3 = pd.json_normalize(data=z_for_sale_resp_json3['props'])

df_z_for_sale = pd.concat([df_z_for_sale1,df_z_for_sale2,df_z_for_sale3], ignore_index=True)

print('Num of rows:', len(df_z_for_sale))
print('Num of cols:', len(df_z_for_sale.columns))
#df_z_for_sale.head(50)

Num of rows: 50
Num of cols: 29


In [11]:
# download file (if needed)
df_z_for_sale=df_z_for_sale.sort_values('daysOnZillow')
df_z_for_sale.to_csv('df_z_for_sale.csv')
files.download('df_z_for_sale.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import pandas as pd
import requests
import json
import urllib
import io

In [13]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

In [14]:
df=df_z_for_sale

In [15]:
df["loc"] = df['address'].apply(geolocator.geocode)

In [16]:
df.dropna(subset=['loc'],inplace=True)

In [17]:
df["price2"] = df['price'].apply(lambda x: f'{x/10**6:.1f}M')

In [18]:
#df['Index'] = str(df.reset_index().index)
#df = df.sort_values('daysOnZillow')
df['Index'] = df.reset_index().index.astype(str)

In [19]:
df["address_price"] = df['Index'] + ', ' + df['address'] + ', ' + df['price2']

In [20]:
df["point"]= df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)

In [21]:
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)

In [22]:
# import the library and its Marker clusterization service
import folium
from folium.plugins import MarkerCluster
# Create a map object and center it to the avarage coordinates to m
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=2)
# if the points are too close to each other, cluster them, create a cluster overlay with MarkerCluster, add to m
marker_cluster = MarkerCluster().add_to(m)

# Custom HTML template for the tooltip
tooltip_html = '''
<div class="tooltip"><input type="text" value="{text}" readonly onclick="this.select(); document.execCommand('copy');"><button onclick="navigator.clipboard.writeText('{text}');">Copy</button></div>
'''

# draw the markers and assign popup and hover texts
# add the markers the the cluster layers so that they are automatically clustered
for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    folium.Marker(location=location,
                  tooltip=r['address_price'])\
    .add_to(marker_cluster)
# display the map
m


In [23]:
df

,dateSold,propertyType,lotAreaValue,address,variableData,priceChange,zestimate,imgSrc,price,detailUrl,bedrooms,contingentListingType,longitude,latitude,listingStatus,zpid,rentZestimate,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,listingSubType.is_foreclosure,listingSubType.is_bankOwned,listingSubType.is_forAuction,loc,price2,Index,address_price,point,lat,lon,altitude
20,None,SINGLE_FAMILY,6400.000000,"6933 Simson St, Oakland, CA 94605",None,NaN,NaN,https://photos.zillowstatic.com/fp/abd15f77382...,899999,/homedetails/6933-Simson-St-Oakland-CA-94605/2...,3,None,-122.170616,37.772823,FOR_SALE,24793646,NaN,0,4,1904,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(6933, Simson Street, Eastmont, Oakland, Alame...",0.9M,0,"0, 6933 Simson St, Oakland, CA 94605, 0.9M","(37.77287765, -122.17065583142202, 0.0)",37.772878,-122.170656,0.0
16,None,SINGLE_FAMILY,4400.000000,"2708 60th Ave, Oakland, CA 94605",None,NaN,969300.0,https://photos.zillowstatic.com/fp/ed36165ebbc...,905000,/homedetails/2708-60th-Ave-Oakland-CA-94605/24...,4,None,-122.188110,37.771790,FOR_SALE,24788169,3828.0,1,2,1540,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(2708, 60th Avenue, Seminary Park, Oakland, Al...",0.9M,1,"1, 2708 60th Ave, Oakland, CA 94605, 0.9M","(37.77180975, -122.18811842292632, 0.0)",37.771810,-122.188118,0.0
24,None,MULTI_FAMILY,4791.600000,"2939 68th Ave, Oakland, CA 94605",None,NaN,NaN,https://photos.zillowstatic.com/fp/dfb15eedcb6...,995000,/homedetails/2939-68th-Ave-Oakland-CA-94605/24...,10,None,-122.177300,37.770590,FOR_SALE,24790653,3274.0,4,6,2992,USA,USD,sqft,True,True,True,NaN,NaN,NaN,"(2939;2941, 68th Avenue, Eastmont, Oakland, Al...",1.0M,2,"2, 2939 68th Ave, Oakland, CA 94605, 1.0M","(37.7705761, -122.1772542088843, 0.0)",37.770576,-122.177254,0.0
1,None,SINGLE_FAMILY,8276.400000,"1738 Indian Way, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/5eaa3aa1231...,1095000,/homedetails/1738-Indian-Way-Oakland-CA-94611/...,3,None,-122.207370,37.840214,FOR_SALE,24817631,NaN,4,2,1584,USA,USD,sqft,True,True,True,NaN,NaN,NaN,"(1738, Indian Way, Oakland, Alameda County, Ca...",1.1M,3,"3, 1738 Indian Way, Oakland, CA 94611, 1.1M","(37.8402173, -122.2073642704905, 0.0)",37.840217,-122.207364,0.0
40,None,SINGLE_FAMILY,5967.720000,"6 Wyman Pl, Oakland, CA 94619",None,-70000.0,1033900.0,https://photos.zillowstatic.com/fp/6ff9a949df1...,925000,/homedetails/6-Wyman-Pl-Oakland-CA-94619/24784...,3,None,-122.188540,37.782100,FOR_SALE,24784149,3227.0,4,2,1800,USA,USD,sqft,True,True,True,NaN,NaN,NaN,"(6, Wyman Place, Maxwell Park, Oakland, Alamed...",0.9M,4,"4, 6 Wyman Pl, Oakland, CA 94619, 0.9M","(37.78211895, -122.18854719286996, 0.0)",37.782119,-122.188547,0.0
19,None,TOWNHOUSE,2178.000000,"33 Graeagle, Oakland, CA 94605",None,NaN,930700.0,https://photos.zillowstatic.com/fp/57b7eb655d9...,875000,/homedetails/33-Graeagle-Oakland-CA-94605/2480...,3,None,-122.129890,37.770187,FOR_SALE,24809464,4010.0,4,3,1636,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(33, Graeagle, Oakland, Alameda County, Califo...",0.9M,5,"5, 33 Graeagle, Oakland, CA 94605, 0.9M","(37.77016795, -122.12994189712765, 0.0)",37.770168,-122.129942,0.0
42,None,SINGLE_FAMILY,4356.000000,"3374 Madera Ave, Oakland, CA 94619",None,NaN,NaN,https://photos.zillowstatic.com/fp/5bdd4370f52...,998000,/homedetails/3374-Madera-Ave-Oakland-CA-94619/...,4,None,-122.191450,37.781890,FOR_SALE,299067815,4526.0,5,3,1657,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(3374, Madera Avenue, Maxwell Park, Oakland, A...",1.0M,6,"6, 3374 Madera Ave, Oakland, CA 94619, 1.0M","(37.781907700000005, -122.19144944555576, 0.0)",37.781908,-122.191449,0.0
8,None,SINGLE_FAMILY,5508.000000,"8040 Shepherd Canyon Rd, Oakland, CA 94611",None,NaN,1091200.0,https://photos.zillowstatic.com/fp/8c89a12e734...,959900,/homedetails/8040-Shepherd-Canyon-Rd-Oakland-C...,4,None,-122.185860,37.835740,FOR_SALE,24815416,5153.0,5,2,1778,USA,USD,sqft,True,True,NaN,NaN,N

In [22]:
#https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6#5352

# End Notebook